<a href="https://colab.research.google.com/github/pea-sys/Til/blob/master/PyTorch%E6%9C%AC7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 7.1 単語分割（Janome、MeCab+NEologd）
本ファイルでは、JanomeもしくはMeCab+NEologdを使用して分かち書きします
※　本章のファイルはすべてUbuntuでの動作を前提としています。Windowsなど文字コードが違う環境での動作にはご注意下さい。

# 7.1 学習目標
機械学習における自然言語処理の流れを理解する
JanomeおよびMeCab+NEologdを用いた形態素解析を実装できるようになる

[写計元](https://github.com/YutaroOgawa/pytorch_advanced/blob/master/7_nlp_sentiment_transformer/7-1_Tokenizer.ipynb)

In [0]:
import os
import urllib.request
import zipfile
import tarfile

In [0]:
# フォルダ「data」が存在しない場合は作成する
data_dir = "./data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

# word2vec学習済みモデルをダウンロード

In [0]:
# word2vecの日本語学習済みモデル（東北大学 乾・岡崎研究室）をダウンロード。時間が15分ほどかかります

url = "http://www.cl.ecei.tohoku.ac.jp/~m-suzuki/jawiki_vector/data/20170201.tar.bz2"
save_path = "./data/20170201.tar.bz2"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# './data/20170201.tar.bz2'の解凍　5分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('./data/20170201.tar.bz2', 'r|bz2')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「entity_vector」というものができ、その中に「entity_vector.model.bin」というファイルができています。

# fastTextの英語学習済みモデルをダウンロード

In [0]:

# fastTextの公式の英語学習済みモデル（650MB）をダウンロード。時間が5分ほどかかります
url = "https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip"
save_path = "./data/wiki-news-300d-1M.vec.zip"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# フォルダ「data」内の「/wiki-news-300d-1M.vec.zip」を解凍する

zip = zipfile.ZipFile("./data/wiki-news-300d-1M.vec.zip")
zip.extractall("./data/")  # ZIPを解凍
zip.close()  # ZIPファイルをクローズ

# フォルダ「data」内にフォルダ「wiki-news-300d-1M.vec」というものができます。

# IMDbデータセットをダウンロード

In [0]:
# IMDbデータセットをダウンロード。30秒ほどでダウンロードできます

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
save_path = "./data/aclImdb_v1.tar.gz"
if not os.path.exists(save_path):
    urllib.request.urlretrieve(url, save_path)

In [0]:
# './data/aclImdb_v1.tar.gz'の解凍　1分ほどかかります

# tarファイルを読み込み
tar = tarfile.open('./data/aclImdb_v1.tar.gz')
tar.extractall('./data/')  # 解凍
tar.close()  # ファイルをクローズ

# フォルダ「data」内にフォルダ「aclImdb」というものができます。

fastTextの学習済みモデルをダウンロード

In [0]:
!pip install janome

     |████████████████████████████████| 21.5MB 1.3MB/s 


In [0]:
from janome.tokenizer import Tokenizer

j_t = Tokenizer()

text = '機械学習が好きです。'

for token in j_t.tokenize(text):
    print(token)

機械	名詞,一般,*,*,*,*,機械,キカイ,キカイ
学習	名詞,サ変接続,*,*,*,*,学習,ガクシュウ,ガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [0]:
# 単語分割する関数を定義


def tokenizer_janome(text):
    return [tok for tok in j_t.tokenize(text, wakati=True)]


text = '機械学習が好きです。'
print(tokenizer_janome(text))

['機械', '学習', 'が', '好き', 'です', '。']


In [0]:
!cat /etc/issue

Ubuntu 18.04.3 LTS \n \l



In [0]:
!sudo apt install file

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmagic-mgc libmagic1
The following NEW packages will be installed:
  file libmagic-mgc libmagic1
0 upgraded, 3 newly installed, 0 to remove and 16 not upgraded.
Need to get 275 kB of archives.
After this operation, 5,294 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.3 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5.32-2ubuntu0.3 [68.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 file amd64 1:5.32-2ubuntu0.3 [22.1 kB]
Fetched 275 kB in 1s (389 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program i

# 1.2 MeCab

In [0]:
#MeCabのインストール
!sudo apt install mecab
!sudo apt install libmecab-dev
!sudo apt install mecab-ipadic-utf8

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libmecab2 mecab-jumandic mecab-jumandic-utf8 mecab-utils
The following NEW packages will be installed:
  libmecab2 mecab mecab-jumandic mecab-jumandic-utf8 mecab-utils
0 upgraded, 5 newly installed, 0 to remove and 16 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 219 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libmecab2 amd64 0.996-5 [257 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-utils amd64 0.996-5 [4,856 B]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic-utf8 all 7.0-20130310-4 [16.2 MB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 mecab-jumandic

In [0]:
!git clone https://github.com/neologd/mecab-ipadic-neologd.git
%cd mecab-ipadic-neologd
!sudo bin/install-mecab-ipadic-neologd

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 9143, done.
remote: Total 9143 (delta 0), reused 0 (delta 0), pack-reused 9143
Receiving objects: 100% (9143/9143), 413.05 MiB | 57.23 MiB/s, done.
Resolving deltas: 100% (5270/5270), done.
/content/mecab-ipadic-neologd
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-

In [0]:
! git clone https://github.com/neologd/mecab-ipadic-neologd.git
% cd mecab-ipadic-neologd
!./bin/install-mecab-ipadic-neologd -n

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 9143, done.
remote: Total 9143 (delta 0), reused 0 (delta 0), pack-reused 9143
Receiving objects: 100% (9143/9143), 413.05 MiB | 54.79 MiB/s, done.
Resolving deltas: 100% (5270/5270), done.
/content/mecab-ipadic-neologd/mecab-ipadic-neologd
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[

In [0]:
os.listdir()

['libexec',
 'diff',
 'bin',
 'build',
 'misc',
 'COPYING',
 'README.ja.md',
 '.git',
 '.travis.yml',
 '.github',
 'seed',
 'README.md',
 'ChangeLog']

In [0]:
!conda install -c anaconda swig
!pip install mecab-python3

/bin/bash: conda: command not found
     |████████████████████████████████| 17.1MB 200kB/s 


In [0]:
import MeCab

m_t = MeCab.Tagger('-Ochasen')

text = '機械学習が好きです。'

print(m_t.parse(text))

機械	キカイ	機械	名詞-一般		
学習	ガクシュウ	学習	名詞-サ変接続		
が	ガ	が	助詞-格助詞-一般		
好き	スキ	好き	名詞-形容動詞語幹		
です	デス	です	助動詞	特殊・デス	基本形
。	。	。	記号-句点		
EOS



In [0]:
import subprocess

cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
path = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                           shell=True).communicate()[0]).decode('utf-8')

In [0]:
import MeCab

m_t = MeCab.Tagger("-d {0}".format(path))#MeCab.Tagger('-Ochasen -d /mecab-ipadic-neologd')

text = '機械学習が好きです。'

print(m_t.parse(text))

機械学習	名詞,固有名詞,一般,*,*,*,機械学習,キカイガクシュウ,キカイガクシュー
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
好き	名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。
EOS



In [0]:
# 単語分割する関数を定義

m_t =  MeCab.Tagger("-d {0}".format(path))#MeCab.Tagger('-Owakati -d /usr/lib/mecab/dic/mecab-ipadic-neologd')


def tokenizer_mecab(text):
    text = m_t.parse(text)  # これでスペースで単語が区切られる
    ret = text.strip().split()  # スペース部分で区切ったリストに変換
    return ret


text = '機械学習が好きです。'
print(tokenizer_mecab(text))

['機械学習', '名詞,固有名詞,一般,*,*,*,機械学習,キカイガクシュウ,キカイガクシュー', 'が', '助詞,格助詞,一般,*,*,*,が,ガ,ガ', '好き', '名詞,形容動詞語幹,*,*,*,*,好き,スキ,スキ', 'です', '助動詞,*,*,*,特殊・デス,基本形,です,デス,デス', '。', '記号,句点,*,*,*,*,。,。,。', 'EOS']
